In [1]:
import pandas as pd
import glob
import numpy as np
from tsfresh import extract_features, extract_relevant_features
%matplotlib inline

import os
cdir = os.getcwd()
print(cdir)
os.chdir(cdir + '/train/train')
print(os.getcwd())

N:\activity-recognition-abc
N:\activity-recognition-abc\train\train


In [2]:
def process(sensordata, timestamps):
    # process
    
    return sensordata

In [3]:
mocap_axis = []
for xyz in range(1,30):
    mocap_axis.append('X'+str(xyz))
    mocap_axis.append('Y'+str(xyz))
    mocap_axis.append('Z'+str(xyz))
print(mocap_axis)

['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'X3', 'Y3', 'Z3', 'X4', 'Y4', 'Z4', 'X5', 'Y5', 'Z5', 'X6', 'Y6', 'Z6', 'X7', 'Y7', 'Z7', 'X8', 'Y8', 'Z8', 'X9', 'Y9', 'Z9', 'X10', 'Y10', 'Z10', 'X11', 'Y11', 'Z11', 'X12', 'Y12', 'Z12', 'X13', 'Y13', 'Z13', 'X14', 'Y14', 'Z14', 'X15', 'Y15', 'Z15', 'X16', 'Y16', 'Z16', 'X17', 'Y17', 'Z17', 'X18', 'Y18', 'Z18', 'X19', 'Y19', 'Z19', 'X20', 'Y20', 'Z20', 'X21', 'Y21', 'Z21', 'X22', 'Y22', 'Z22', 'X23', 'Y23', 'Z23', 'X24', 'Y24', 'Z24', 'X25', 'Y25', 'Z25', 'X26', 'Y26', 'Z26', 'X27', 'Y27', 'Z27', 'X28', 'Y28', 'Z28', 'X29', 'Y29', 'Z29']


In [4]:
import os
os.sep

'\\'

In [5]:
# turning tensorflow warnings off

In [6]:
# there are 3 subjects
subject1 = {}
subject2 = {}
subject3 = {}

# we will load every single right arm data, separate based on subject id, do feature extraction, run t-SNE

all_sensors = ['right_arm', 'right_wrist', 'left_hip', 'left_wrist'] #, 'mocap']

data_folder = [f'{sensor}/*.csv' for sensor in all_sensors]

print(data_folder)

files = glob.glob(data_folder[0])

# processing will be done as numpy array

subject1['data'] = []
subject2['data'] = []
subject3['data'] = []
 
# data format : data -> id, process(timeseries, timestamps) : right_arm as numpy array (missing data will be imputed)


# id is generated by concatenating INT(subject+trial)

for f in files:
    #print(f)
    c_sub = {} # current subject
    if f.split(os.sep)[1].split('_')[0] == 'subject1':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            #print(sensor)
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                
        subject1['data'].append(c_sub)

            
    elif f.split(os.sep)[1].split('_')[0] == 'subject2':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                
        subject2['data'].append(c_sub)
            
    elif f.split(os.sep)[1].split('_')[0] == 'subject3':
        
        c_sub['id'] = int(f.split(os.sep)[1].split('_')[0][-1] + f.split(os.sep)[1].split('_')[2].split('.')[0])
        #print(ra['id'])
        
        for sensor in all_sensors:
            ra = pd.read_csv(sensor + os.sep + f.split(os.sep)[1])
            #print(ra)
            ra.sort_values(by=['timestamp'], inplace = True)
            c_sub[sensor] = {}
            if sensor == 'mocap':
                for axis in mocap_axis:
                    try:
                        c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
                    except:
                        c_sub[sensor][axis] = [0]
            else:
                for axis in ['X', 'Y', 'Z']:
                    c_sub[sensor][axis] = process(ra[axis], ra['timestamp'])
        
        subject3['data'].append(c_sub)
    else:
        print('either new subject or a bug')
        
        
labels = pd.read_csv("labels.txt", sep=' ', header=None)
print(labels.head())
labels = labels[0].str.split(",", n=2, expand=True)
labels.columns = ['file_id', 'macro', 'micro'] #give names to the columns
labels.index = labels['file_id'] #use the file id as index to make it searchable by file_id
print(labels.head())

# label generation
subject1['label_mac'] = {}
subject2['label_mac'] = {}
subject3['label_mac'] = {}

subject1['label_mic'] = {}
subject2['label_mic'] = {}
subject3['label_mic'] = {}

for i in range(len(labels)):
    #print(labels.iloc[i]['file_id'])
    #print(labels.iloc[i]['file_id'].split('_')[0][-1])
    tid = int(labels.iloc[i]['file_id'].split('_')[0][-1] + labels.iloc[i]['file_id'].split('_')[-1])
    #print(tid)
    label = labels.iloc[i]['macro']
    label_mic = labels.iloc[i]['micro'].split(',')[:-1]
    #print(label)
    if labels.iloc[i]['file_id'].split('_')[0][-1] == '1':
        subject1['label_mac'][tid] = label
        subject1['label_mic'][tid] = label_mic
    elif labels.iloc[i]['file_id'].split('_')[0][-1] == '2':
        subject2['label_mac'][tid] = label
        subject2['label_mic'][tid] = label_mic
    elif labels.iloc[i]['file_id'].split('_')[0][-1] == '3':
        subject3['label_mac'][tid] = label
        subject3['label_mic'][tid] = label_mic
    else:
        print('some bug')
        
        
# re-formatting dataset for training

X = []
y = []
y_ml = []  # multi-label

for i in range(len(subject1['data'])):
    tid = subject1['data'][i]['id']
    y.append(subject1['label_mac'][tid])
    y_ml.append(subject1['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject1['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject1['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject1['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X.append(cs_data)
    
    
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y)

y_ohe = to_categorical(vec,len(set(vec)))

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_ml_ohe = mlb.fit_transform(y_ml)

# validatin on subject2, 3

# re-formatting dataset for training

X2 = []
y2 = []
y_ml2 = []

for i in range(len(subject2['data'])):
    tid = subject2['data'][i]['id']
    y2.append(subject2['label_mac'][tid])
    y_ml2.append(subject2['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject2['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject2['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject2['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X2.append(cs_data)
    
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y2)

y2_ohe = to_categorical(vec,len(set(vec)))

mlb = MultiLabelBinarizer()
y_ml2_ohe = mlb.fit_transform(y_ml2)

# validatin on subject2, 3

# re-formatting dataset for training

X3 = []
y3 = []
y_ml3 = []

for i in range(len(subject3['data'])):
    tid = subject3['data'][i]['id']
    y3.append(subject3['label_mac'][tid])
    y_ml3.append(subject3['label_mic'][tid])
    # X shape -> [ip1, ip2, ip3, ip4] ip1 = (80, len, channel) -> 4, 80, len

for sensor in all_sensors:
    cs_data = []
    for i in range(len(subject3['data'])):
        sub_data = []
        if sensor == 'mocap':
            for sig in mocap_axis:
                sub_data.append(np.array(subject3['data'][i][sensor][sig]))
        else:
            for sig in ['X', 'Y', 'Z']:
                sub_data.append(np.array(subject3['data'][i][sensor][sig]))
        sub_data = np.array(sub_data)
        # print(sub_data.shape)
        sub_data = np.swapaxes(sub_data, 0, 1)
        cs_data.append(sub_data)
    X3.append(cs_data)
    

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y3)

y3_ohe = to_categorical(vec,len(set(vec)))

mlb = MultiLabelBinarizer()
y_ml3_ohe = mlb.fit_transform(y_ml3)

['right_arm/*.csv', 'right_wrist/*.csv', 'left_hip/*.csv', 'left_wrist/*.csv']
                                           0
0           subject2_file_457,sandwich,Take,
1      subject2_file_679,sandwich,Wash,Take,
2        subject2_file_95,sandwich,Cut,Wash,
3  subject2_file_899,sandwich,other,Cut,Put,
4            subject2_file_368,sandwich,Put,
                             file_id     macro           micro
file_id                                                       
subject2_file_457  subject2_file_457  sandwich           Take,
subject2_file_679  subject2_file_679  sandwich      Wash,Take,
subject2_file_95    subject2_file_95  sandwich       Cut,Wash,
subject2_file_899  subject2_file_899  sandwich  other,Cut,Put,
subject2_file_368  subject2_file_368  sandwich            Put,


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, LSTM, Bidirectional, Input, GlobalMaxPooling1D, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [31]:
# CNN-head
# extracting time-series features with LSTM, then applying CNN on those
def create_model():

    n_sensors = 4 #5
    n_filters = [3, 3, 3, 3] # , 3*29] # X, Y, Z


    # make the input layers
    ips = []

    n_f = 0
    for sensor in all_sensors:
        ips.append(Input(shape = (None, n_filters[n_f]), name = sensor + '_ipX'))
        n_f += 1

    print(f'Input tensors: {ips}')
    
    # lstm feature extractor
    lstms = []
    for ip in ips:
        lstm = LSTM(units = 16, return_sequences = True, dropout = 0.4, recurrent_dropout = 0.25, name = ip.name.split(':')[0] + '_lstm')(ip)
        lstms.append(lstm)

    convs = []
    for lstm in lstms:
        conv = Conv1D(filters = 16, kernel_size = 3, padding = 'same', activation = None, name = lstm.name.split(':')[0] + '_conv')(lstm)
        convs.append(conv)
        
    # batch-normalization
    bns = []
    for conv in convs:
        bn = BatchNormalization()(conv)
        bns.append(bn)

    convs2 = []
    for bn in bns:
        conv2 = Conv1D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu')(bn)
        convs2.append(conv2)
        
    # batch-normalization
    bns2 = []
    for conv2 in convs2:
        bn2 = BatchNormalization()(conv2)
        bns2.append(bn2)



    # concatenation

    concat = Concatenate(axis=1, name = 'concat')(bns2)

    concat_conv = Conv1D(filters = 16, kernel_size = 1, padding = 'same', activation = None, name = 'concat_conv')(concat)
    
    bn3 = BatchNormalization()(concat_conv)
    con_conv2 = Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu', name = 'con_conv2')(bn3)
    bn4 = BatchNormalization()(con_conv2)
    gmp = GlobalMaxPooling1D()(bn4)

    # simple FC
    final_softmax = Dense(3, activation = 'softmax')(gmp)
    simple_conv = Model(ips, final_softmax)
    return simple_conv

In [32]:
simple_lstm_cnn = create_model()

Input tensors: [<tf.Tensor 'right_arm_ipX_5:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'right_wrist_ipX_5:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_hip_ipX_5:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_wrist_ipX_5:0' shape=(None, None, 3) dtype=float32>]


In [33]:
simple_lstm_cnn.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
right_arm_ipX (InputLayer)      [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
right_wrist_ipX (InputLayer)    [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_hip_ipX (InputLayer)       [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
left_wrist_ipX (InputLayer)     [(None, None, 3)]    0                                            
____________________________________________________________________________________________

In [24]:
simple_lstm_cnn.compile(loss='categorical_crossentropy', metrics=['acc', 'mae'], optimizer=Adam(lr=0.001))

In [29]:
def train(simple_conv, X, y_ohe, EPOCH, train_split=1.0):
    # single zero padding for NULL datas

    for i in range(len(X)):
        for j in range(len(X[0])):
            if len(X) == 5 and i == len(X)-1: # assuming mocap always comes in the last index
                if len(X[i][j]) == 0:
                    X[i][j] = np.zeros((1, 87))
            else:
                if len(X[i][j]) == 0:
                    X[i][j] = np.array([[0, 0, 0]])

    report_acc = 0.0
    for epoch in range(EPOCH):
        print(f'Training epoch {epoch} ...')
        avg_loss = 0.0
        avg_acc = 0.0
        avg_mae = 0.0

        for sample_i in tqdm(range(int(len(X[0])*train_split))):
            [train_loss, acc, mae]   =  simple_conv.train_on_batch([X[0][sample_i].reshape(1,len(X[0][sample_i]),3), 
                                        X[1][sample_i].reshape(1,len(X[1][sample_i]),3), 
                                        X[2][sample_i].reshape(1,len(X[2][sample_i]),3), 
                                        X[3][sample_i].reshape(1,len(X[3][sample_i]),3)],
                                        #X[4][sample_i].reshape(1,len(X[4][sample_i]),3*29),], 
                                        y_ohe[sample_i].reshape(1,3))
            avg_loss += train_loss/(len(X[0])*train_split)
            avg_acc += acc/(len(X[0])*train_split)
            avg_mae += mae/(len(X[0])*train_split)
        report_acc = avg_acc
        print(f'acc: {avg_acc} mae: {avg_mae} loss: {avg_loss}')
        #print('Running validation ...')
        avg_loss = 0.0
        avg_acc = 0.0
        avg_mae = 0.0
        for sample_i in tqdm(range(int(len(X[0])*train_split), len(X[0]))):
            [test_loss, acc, mae]   =  simple_conv.test_on_batch([X[0][sample_i].reshape(1,len(X[0][sample_i]),3), 
                                        X[1][sample_i].reshape(1,len(X[1][sample_i]),3), 
                                        X[2][sample_i].reshape(1,len(X[2][sample_i]),3), 
                                        X[3][sample_i].reshape(1,len(X[3][sample_i]),3)],
                                        #X[4][sample_i].reshape(1,len(X[4][sample_i]),3*29)], 
                                        y_ohe[sample_i].reshape(1,3))
            avg_loss += test_loss/(len(X[0])*(1-train_split))
            avg_acc += acc/(len(X[0])*(1-train_split))
            avg_mae += mae/(len(X[0])*(1-train_split))
            
        #print(f'test acc: {avg_acc} mae: {avg_mae} loss: {avg_loss}')
        
    
    return simple_conv, report_acc

In [30]:
def test(simple_conv, X2, y2_ohe):
    for i in range(len(X2)):
        for j in range(len(X2[0])):
            if len(X2) == 5 and i == len(X2)-1: # assuming mocap always comes in the last index
                if len(X2[i][j]) == 0:
                    X2[i][j] = np.zeros((1, 87))
            else:
                if len(X2[i][j]) == 0:
                    X2[i][j] = np.array([[0, 0, 0]])

    print('Running validation on subject ...')
    avg_loss = 0.0
    avg_acc = 0.0
    avg_mae = 0.0
    for sample_i in tqdm(range(int(len(X2[0])))):
        [test_loss, acc, mae]   =  simple_conv.test_on_batch([X2[0][sample_i].reshape(1,len(X2[0][sample_i]),3), 
                                    X2[1][sample_i].reshape(1,len(X2[1][sample_i]),3), 
                                    X2[2][sample_i].reshape(1,len(X2[2][sample_i]),3), 
                                    X2[3][sample_i].reshape(1,len(X2[3][sample_i]),3)],
                                    #X2[4][sample_i].reshape(1,len(X2[4][sample_i]),3*29)], 
                                    y2_ohe[sample_i].reshape(1,3))
        avg_loss += test_loss/(len(X2[0]))
        avg_acc += acc/(len(X2[0]))
        avg_mae += mae/(len(X2[0]))
    print(f'test acc: {avg_acc} mae: {avg_mae} loss: {avg_loss}')
    return avg_acc

In [34]:
# train on subject1
# validate on subject2, 3

from tqdm import tqdm

report = []

for xytst in [(X, y_ohe, 'X1'), (X2, y2_ohe, 'X2'), (X3, y3_ohe, 'X3')]:
    
    simple_lstm_cnn = create_model()
    simple_lstm_cnn.compile(loss='categorical_crossentropy', metrics=['acc', 'mae'], optimizer=Adam(lr=0.001))
    
    xytr = [a for a in [(X, y_ohe), (X2, y2_ohe), (X3, y3_ohe)] if a!=xytst]

    EPOCH = 2
    train_split = 1.0

    simple_lstm_cnn, acc = train(simple_lstm_cnn, xytr[0][0], xytr[0][1], EPOCH, train_split)

    simple_lstm_cnn, acc2 = train(simple_lstm_cnn, xytr[1][0], xytr[1][1], EPOCH, train_split)

    print('-------------------------------------------------')
    print('Test phase')
    print(f'Testing {xytst[2]}')

    t_acc = test(simple_lstm_cnn, xytst[0], xytst[1])
    report.append(f'acc1: {acc} acc2: {acc2} test_acc: {t_acc}')


print('--------------------report----------------------')
for ta in report:
    print(ta)
    

Input tensors: [<tf.Tensor 'right_arm_ipX_6:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'right_wrist_ipX_6:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_hip_ipX_6:0' shape=(None, None, 3) dtype=float32>, <tf.Tensor 'left_wrist_ipX_6:0' shape=(None, None, 3) dtype=float32>]




  0%|                                                                                                 | 0/80 [00:00<?, ?it/s]

Training epoch 0 ...




  1%|█                                                                                        | 1/80 [00:15<20:11, 15.33s/it]

  2%|██▏                                                                                      | 2/80 [00:31<20:13, 15.56s/it]

  4%|███▎                                                                                     | 3/80 [00:40<17:34, 13.69s/it]

  5%|████▍                                                                                    | 4/80 [00:56<18:15, 14.42s/it]

  6%|█████▌                                                                                   | 5/80 [01:04<15:36, 12.48s/it]

  8%|██████▋                                                                                  | 6/80 [01:19<16:14, 13.16s/it]

  9%|███████▊                                                                                 | 7/80 [01:21<11:49,  9.71s/it]

 10%|████████▉                                                                                | 8/80 [01:29<1

 78%|████████████████████████████████████████████████████████████████████▏                   | 62/80 [10:34<02:43,  9.07s/it]

 79%|█████████████████████████████████████████████████████████████████████▎                  | 63/80 [10:40<02:19,  8.21s/it]

 80%|██████████████████████████████████████████████████████████████████████▍                 | 64/80 [10:50<02:20,  8.77s/it]

 81%|███████████████████████████████████████████████████████████████████████▌                | 65/80 [10:59<02:09,  8.64s/it]

 82%|████████████████████████████████████████████████████████████████████████▌               | 66/80 [11:07<01:59,  8.51s/it]

 84%|█████████████████████████████████████████████████████████████████████████▋              | 67/80 [11:20<02:10, 10.02s/it]

 85%|██████████████████████████████████████████████████████████████████████████▊             | 68/80 [11:28<01:51,  9.33s/it]

 86%|███████████████████████████████████████████████████████████████████████████▉            | 69/80 [11:35<01:

...

 59%|███████████████████████████████████████████████████▎                                   | 62/105 [06:54<04:47,  6.69s/it]

 60%|████████████████████████████████████████████████████▏                                  | 63/105 [07:03<05:12,  7.45s/it]

 61%|█████████████████████████████████████████████████████                                  | 64/105 [07:05<03:59,  5.85s/it]

 62%|█████████████████████████████████████████████████████▊                                 | 65/105 [07:10<03:46,  5.65s/it]

 63%|██████████████████████████████████████████████████████▋                                | 66/105 [07:15<03:32,  5.44s/it]

 64%|███████████████████████████████████████████████████████▌                               | 67/105 [07:20<03:22,  5.33s/it]

 65%|████████████████████████████████████████████████████████▎                              | 68/105 [07:27<03:31,  5.70s/it]

 66%|█████████████████████████████████████████████████████████▏                             | 69/105 [07:32<03:

acc: 0.40000000000000024 mae: 0.41560868843059456 loss: 2.1844053121328004
-------------------------------------------------
Test phase
Testing X3
Running validation on subject ...




  1%|▊                                                                                       | 1/103 [00:02<03:59,  2.34s/it]

  2%|█▋                                                                                      | 2/103 [00:03<03:13,  1.92s/it]

  3%|██▌                                                                                     | 3/103 [00:07<04:31,  2.71s/it]

  4%|███▍                                                                                    | 4/103 [00:11<04:57,  3.01s/it]

  5%|████▎                                                                                   | 5/103 [00:13<04:36,  2.82s/it]

  6%|█████▏                                                                                  | 6/103 [00:16<04:17,  2.65s/it]

  7%|█████▉                                                                                  | 7/103 [00:19<04:40,  2.93s/it]

  8%|██████▊                                                                                 | 8/103 [00:23<0

 60%|████████████████████████████████████████████████████▎                                  | 62/103 [03:02<02:02,  2.98s/it]

 61%|█████████████████████████████████████████████████████▏                                 | 63/103 [03:03<01:39,  2.50s/it]

 62%|██████████████████████████████████████████████████████                                 | 64/103 [03:07<01:50,  2.83s/it]

 63%|██████████████████████████████████████████████████████▉                                | 65/103 [03:10<01:48,  2.85s/it]

 64%|███████████████████████████████████████████████████████▋                               | 66/103 [03:13<01:55,  3.13s/it]

 65%|████████████████████████████████████████████████████████▌                              | 67/103 [03:16<01:43,  2.89s/it]

 66%|█████████████████████████████████████████████████████████▍                             | 68/103 [03:21<02:05,  3.57s/it]

 67%|██████████████████████████████████████████████████████████▎                            | 69/103 [03:25<02:

test acc: 0.3883495145631067 mae: 0.4174591458483952 loss: 1.8612787761155845
--------------------report----------------------
acc1: 0.48750000000000027 acc2: 0.4095238095238098 test_acc: 0.47500000000000026
acc1: 0.46250000000000024 acc2: 0.44761904761904797 test_acc: 0.3904761904761907
acc1: 0.5874999999999999 acc2: 0.40000000000000024 test_acc: 0.3883495145631067


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras_lr_multiplier import LRMultiplier

model = Sequential()
model.add(Dense(
    units=5,
    input_shape=(5,),
    activation='tanh',
    name='Dense',
))
model.add(Dense(
    units=2,
    activation='softmax',
    name='Output',
))
model.compile(
    optimizer=LRMultiplier('adam', {'Output': 1.5}),
    loss='sparse_categorical_crossentropy',
)

ValueError: ('Could not interpret optimizer identifier:', <keras_lr_multiplier.multiplier.LRMultiplier object at 0x000001A68916B9B0>)